# Template Notebook for NSGA II Multi-Objective-Optimization exploring the Prompt Embedding space
Notebook Version: 0.34 (22/03/2024)
* use NSGATournamentSelector instead of TournamentSelector
* tuple metrics in chart only display first element

## Google Colab Setup

In [ ]:
# Google Colab: Execute this to install packages and setup drive
!pip install "evolutionary[all] @ git+https://git@github.com/malthee/evolutionary-diffusion.git"

In [ ]:
# Mount drive to save results
from google.colab import drive
import evolutionary_imaging.processing as ip
drive.mount("/content/drive")
base_path = "/content/drive/MyDrive/evolutionary/"
ip.RESULTS_FOLDER = base_path + ip.RESULTS_FOLDER

In [ ]:
# Check if GPU is available
import torch
print(torch.cuda.is_available())

## Project Setup

In [ ]:
from evolutionary.plotting import plot_fitness_statistics
import evolutionary_imaging.processing as ip
from diffusers.utils import logging
from evolutionary_imaging.processing import create_animation_from_generations, create_generation_radar_chart_grid, save_images_from_generation
import torch
import os

In [ ]:
logging.disable_progress_bar() # Or else your output will be full of progress bars
logging.set_verbosity_error() # Enable again if you are having problems
os.environ["TOKENIZERS_PARALLELISM"] = "false" # To remove warning of libraries using tokenizers
# Change the results folder for images if you want to 
# ip.RESULTS_FOLDER = 'choose_your_destination'

def save_images_post_sort(g, a):
    save_images_from_generation(a.fronts[0], g)  # Only save from first front
    
# Check torch random state, used across all libraries. Caution setting fixed seeds as it affects not only generation but also variation.
print(torch.random.get_rng_state())

In [ ]:
from evolutionary_prompt_embedding.argument_types import PooledPromptEmbedData
from evolutionary_prompt_embedding.image_creation import SDXLPromptEmbeddingImageCreator
from evolutionary_prompt_embedding.variation import PooledArithmeticCrossover, PooledUniformGaussianMutator, UniformGaussianMutatorArguments
from evolutionary.evolutionary_selectors import TournamentSelector
from evolutionary.algorithms.nsga_ii import NSGA_II, NSGATournamentSelector
from evolutionary_imaging.evaluators import AestheticsImageEvaluator, CLIPScoreEvaluator, MultiCLIPIQAEvaluator, \
    SingleCLIPIQAEvaluator
from evolutionary.evolution_base import MultiObjectiveEvaluator
from evolutionary_prompt_embedding.value_ranges import SDXLTurboEmbeddingRange, SDXLTurboPooledEmbeddingRange

population_size = 100
num_generations = 50
batch_size = 1
elitism = None
metrics = ("sharpness", "noisiness", "quality")

embedding_range = SDXLTurboEmbeddingRange()
pooled_embedding_range = SDXLTurboPooledEmbeddingRange()
creator = SDXLPromptEmbeddingImageCreator(batch_size=batch_size, inference_steps=4)
evaluator = MultiCLIPIQAEvaluator(metrics=metrics)
crossover = PooledArithmeticCrossover(interpolation_weight=0.5, interpolation_weight_pooled=0.5)
mutation_arguments = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=2, 
                                                     clamp_range=(embedding_range.minimum, embedding_range.maximum)) 
mutation_arguments_pooled = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=0.4, 
                                                            clamp_range=(pooled_embedding_range.minimum, pooled_embedding_range.maximum))
mutator = PooledUniformGaussianMutator(mutation_arguments, mutation_arguments_pooled)
selector = NSGATournamentSelector()

# Prepare initial arguments
#init_embed = creator.arguments_from_prompt(prompt) # with prompt
#init_args = [init_embed for _ in range(population_size)]
init_args = [PooledPromptEmbedData(embedding_range.random_tensor_in_range(), pooled_embedding_range.random_tensor_in_range()) 
             for _ in range(population_size)] # random

nsga = NSGA_II(
    num_generations=num_generations,
    population_size=population_size,
    solution_creator=creator,
    selector=selector,
    crossover=crossover,
    mutator=mutator,
    evaluator=evaluator,
    elitism_count=elitism,
    initial_arguments=init_args,
    post_non_dominated_sort_callback=save_images_post_sort
)

In [ ]:
best_solution = nsga.run()

In [ ]:
from diffusers.utils import make_image_grid

# Show best solution
print(best_solution.fitness)
make_image_grid(best_solution.result.images, 1, batch_size)

## Visualize the evolution

In [ ]:
for gen in range(num_generations):
    create_generation_radar_chart_grid(gen, tuple(m if isinstance(m, str) else m[0] for m in metrics), 
                                       max_images=2, 
                                       label_padding=12,
                                       max_value=1.0 # Depends on the objectives set above
                                       )

In [ ]:
video_loc = create_animation_from_generations(num_generations)
print(video_loc)

## Plot fitness statistics (Separately)

In [ ]:
plot_fitness_statistics(num_generations, nsga.best_fitness, nsga.worst_fitness, nsga.avg_fitness, title=metrics[1], multi_objective_plot_index=1)

In [ ]:
plot_fitness_statistics(num_generations, nsga.best_fitness, nsga.worst_fitness, nsga.avg_fitness, title=metrics[2], multi_objective_plot_index=2)

## Plot fitness statistics (Together)

In [ ]:
plot_fitness_statistics(num_generations, avg_fitness=nsga.avg_fitness, labels=[m if isinstance(m, str) else " ".join(m) for m in metrics])

## Save notebook and components

In [ ]:
!jupyter nbconvert --to html nsga_notebook.ipynb

### Save the run to disk

In [ ]:
import pickle
import os
from datetime import datetime

os.makedirs("saved_runs", exist_ok=True)
output_file = os.path.join("saved_runs", f"nsga_iqa_oldcolorfulbeautiful{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl")
with open(output_file, "wb") as f:
    pickle.dump(nsga, f)
print(f"Run saved to {output_file}")

### Load the run from disk 
Notebook and library versions should match with the saved run

In [ ]:
import pickle
import os

with open(os.path.join("saved_runs", "insert_filename"), "rb") as f:
    nsga = pickle.load(f)

## Fallback functions for when something went wrong

### Access Best Solution from Disk

In [ ]:
import os
import glob
import evolutionary_imaging.processing as ip
from PIL import Image

num_generations = 4  # Set this to the number of generations you ran (if you didn't finish)
generation_dir = os.path.join("results", f"{num_generations}")
image_files = glob.glob(os.path.join(generation_dir, "*.png"))
image_files.sort(key=ip.fitness_filename_sorting_key, reverse=True)
print(image_files[0])
Image.open(image_files[0])

### ffmpeg is not installed, create GIF instead

In [ ]:
from evolutionary_imaging.processing import create_animation_from_generations_pil
video_loc = create_animation_from_generations_pil(num_generations)
print(video_loc)